# Network Analysis of the Chromatin

In [2]:
# ================================
#           libraries
# ================================

# %pip install -q MDTraj    # leave this uncommented if running it on Google Colab
import mdtraj as md

import pandas as pd
import numpy as np
import scipy.ndimage as nd
import math
from tqdm import tqdm
from random import random
import plotly.graph_objects as go
import plotly.io as pio

import urllib.request
import ssl
from copy import deepcopy


In [ ]:
# Create unverified context for this specific download
ssl_context = ssl._create_unverified_context()

url = 'https://calla.rnet.missouri.edu/genome3d/GSDB/ \\
    Database/AX9716PF/GSE105544_ENCFF010WBP/VC/LorDG/chr1.pdb'

# Download the file
with urllib.request.urlopen(url, context=ssl_context) as response:
    with open('chr1.pdb', 'wb') as out_file:
        out_file.write(response.read())

# Load from local file
chr_str = md.load_pdb('chr1.pdb')
coords = chr_str.xyz.reshape(-1, 3)

### Distance From Center of Mass

In [22]:
com = md.compute_center_of_mass(chr_str)

chr_com = []
for coord in coords:
    distances = np.linalg.norm(com - coord, axis=1)
    min_dist = np.min(distances)
    chr_com.append ([*coord, min_dist])

chr_com = np.array(chr_com)

chr_com.shape


(451, 4)